In [4]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
import sys
import re
import logging
import gensim
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
from preprocess_data import *
config = config_base.config

In [8]:
df = organize_data(config.train_data)
df = df.sample(5000)

In [9]:
df = deal_data(df)

In [10]:
df = shorten_content_all(df, config.max_len)

shorten content, accuracy: 0.9664
max length: 300
min length: 7
mean length:180
median length:176


In [37]:
s = build_answer_range(df)

1093


In [55]:
tmp = df.sample()

In [56]:
merge = tmp['merge'].values[0]
merge

'朝鲜试射洲际弹道导弹 中方表示严重关切和反对。针对有关朝鲜试射“火星-15”型洲际弹道导弹的报道，外交部发言人耿爽29日表示，中方对朝方有关发射活动表示严重关切和反对。 在当日例行记者会上，有记者问，朝中社发布消息称，朝鲜于今天凌晨成功试射“火星-15”型洲际弹道导弹，中方对此有何评论？ 耿爽说，联合国安理会决议对朝鲜利用弹道导弹技术进行发射活动有明确规定。中方对朝方有关发射活动表示严重关切和反对。 “中方强烈敦促朝方遵守安理会相关决议，停止采取加剧半岛紧张的行动。同时希望有关各方慎重行事，共同维护本地区和平稳定。（记者闫子敏）'

In [57]:
answer = tmp['answer'].values[0]
answer

'“火星-15”型洲际弹道导弹'

In [58]:
question = tmp['question'].values[0]
question

'朝中社发布消息称，朝鲜于今天凌晨成功试射了什么导弹'

In [36]:
def build_answer_range(df):
    def match(merge, answer):
        merge_list = jieba.lcut(merge)
        merge_len = len(merge_list)
        answer_list = jieba.lcut(answer)
        answer_len = len(answer_list)
        start = -1
        end = -1
        flag = -1
        for i in range(0, merge_len-answer_len+1):
            if merge_list[i: i+answer_len] == answer_list and flag == -1:
                flag = 0
                continue
            if merge_list[i: i+answer_len] == answer_list and flag == 0:
                flag = 1

        return flag

    merges = df[df['is_in']]['merge'].values
    answers = df[df['is_in']]['answer'].values
    c = 0
    for m, a in zip(merges, answers):
        s = match(m, a)
        if s == 1:
            c += 1
    print(c)
#     answer_range = [match(m, a) for m, a in zip(merges, answers)]

#     start, end = list(zip(*answer_range))
#     df.loc[df['is_in'], 'answer_start'] = start
#     df.loc[df['is_in'], 'answer_end'] = end

#     merge_len = len(merges)
#     right_len = (df['answer_start'] > -1).sum()
#     print('answer generation accuracy: %.4f\n' % (right_len/merge_len))

    return df